In [1]:
import pandas as pd
import json as js
import torch
import cv2
from tqdm import tqdm

In [2]:
sample = pd.read_csv('sample_solution.csv')

In [3]:
sample.head()

,ID_img,region_shape
0,1.JPG,0
1,2.JPG,0
2,3.JPG,0
3,4.JPG,0
4,5.JPG,0


In [4]:
len(sample)

3435

In [5]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best_3.pt')

Using cache found in C:\Users\yeahk/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-11-10 Python-3.9.13 torch-1.13.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


In [6]:
predictions = []
for name in tqdm(sample['ID_img']):
    image = cv2.imread('D:/Study/DigitalBreakThrough2022/CVOmsk/test_dataset_test()/test/'+name)
    predictions.append(model(image))

100%|██████████| 3435/3435 [12:59<00:00,  4.41it/s]


In [7]:
results = []
for tensor in predictions:
    res = []
    if len(tensor.xywh[0]) != 0:
        for obj in tensor.xywh[0].tolist():
            coordinates = {}
            coordinates['cx'] = int(obj[0])
            coordinates['cy'] = int(obj[1])
            coordinates['r'] = int((obj[2] + obj[3]) / 4)
            res.append(str(js.dumps(coordinates)))
    else:
        res = 0
    results.append(res)

In [8]:
sample['region_shape'] = results
sample = sample.set_index('ID_img')
sample.to_csv('submission_4.csv')